In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set_style('white')  # plot formatting
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import validation_curve
from sklearn.metrics import r2_score
from sklearn.model_selection import learning_curve


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

## Logistic Regression
    Letter 1 dataset

In [2]:
#import letter data set
#letter1 for the problem with 'O' as the postive class rest as negative
#letter2, for the problem with A-M as positive and N-Z as negative
letter = pd.read_csv('letter-recognition.data')

letter_ = letter.replace('O', +1) 
letter1 = letter_.replace(to_replace = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 
                                           'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], value = -1)
letter_Y = letter.replace (to_replace = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M'], value = +1) 
letter2 = letter_Y.replace(to_replace = ['N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], value = -1)

X_1 = letter1
Y_1 = X_1[['T']]
X_1 = X_1.iloc[:, 1:]

X_2 = letter2
Y_2 = X_2[['T']]
X_2 = X_2.iloc[:, 1:]

In [4]:
optimal_hyperparameters_1 = []
mean_scores_1 = []
variance_1 = []

for i in range(5):
    #split the data so that training size is 5000
    X_train, X_test, y_train, y_test = train_test_split(X_1, Y_1, train_size = 5000)

    
            # Create a pipeline - RF is a stand in, we will populate the classifier part below
    pipe_log = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

            # Create search space of candidate learning algorithms and their hyperparameters
            # note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
    search_space_log = [{'classifier': [LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]},
                
                {'classifier': [LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]},
                
                {'classifier': [LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}
                ]

            # Create grid search 
    clf_log = GridSearchCV(pipe_log, search_space_log, cv=StratifiedKFold(n_splits=5), 
                   scoring=['accuracy', 'roc_auc', 'f1_micro'], refit=False,
                   verbose=0)

            # Fit grid search
    best_model_log = clf_log.fit(X_train, y_train)

           
            # output best hyperparameter set indexed at best metric scores
    h1 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_accuracy']) ]    
    h2 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_roc_auc']) ]
    h3 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_f1_micro']) ]
    
    optimal_hyperparameters_1.append(h1)
    optimal_hyperparameters_1.append(h2)
    optimal_hyperparameters_1.append(h3)
    mean_scores_1.append(best_model_log.cv_results_['mean_test_accuracy'][np.argmin(best_model_log.cv_results_['rank_test_accuracy'])])
    mean_scores_1.append(best_model_log.cv_results_['mean_test_roc_auc'][np.argmin(best_model_log.cv_results_['rank_test_roc_auc'])])
    mean_scores_1.append(best_model_log.cv_results_['mean_test_f1_micro'][np.argmin(best_model_log.cv_results_['rank_test_f1_micro'])])
    
    variance_1.append(best_model_log.cv_results_['std_test_accuracy'][np.argmin(best_model_log.cv_results_['rank_test_accuracy'])])
    variance_1.append(best_model_log.cv_results_['std_test_roc_auc'][np.argmin(best_model_log.cv_results_['rank_test_roc_auc'])])
    variance_1.append(best_model_log.cv_results_['std_test_f1_micro'][np.argmin(best_model_log.cv_results_['rank_test_f1_micro'])])
print(optimal_hyperparameters_1)
print(mean_scores_1)

[{'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1e-08, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.01, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1e-08, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1e-08, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.01, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1e-08, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1e-08, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier

In [5]:
print(variance_1)

[0.00040000000000000034, 0.007736188381058471, 0.00040000000000000034, 0.0004898979485566361, 0.015093152103589828, 0.0004898979485566361, 0.0, 0.016547797751477553, 0.0, 0.00040000000000000034, 0.014095077310794183, 0.00040000000000000034, 0.00040000000000000034, 0.013705588575289994, 0.00040000000000000034]


## Logistic Regression 
    Letter 2 dataset

In [4]:
optimal_hyperparameters_2 = []
mean_scores_2 = []
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_2, Y_2, train_size = 5000)
    
            # Fit grid search
    best_model_log = clf_log.fit(X_train, y_train)

           
            # output best hyperparameter set indexed at best metric scores
    h1 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_accuracy']) ]    
    h2 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_roc_auc']) ]
    h3 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_f1']) ]
    
    optimal_hyperparameters_2.append(h1)
    optimal_hyperparameters_2.append(h2)
    optimal_hyperparameters_2.append(h3)
    mean_scores_2.append(best_model_log.cv_results_['mean_test_accuracy'][np.argmin(best_model_log.cv_results_['rank_test_accuracy'])])
    mean_scores_2.append(best_model_log.cv_results_['mean_test_roc_auc'][np.argmin(best_model_log.cv_results_['rank_test_roc_auc'])])
    mean_scores_2.append(best_model_log.cv_results_['mean_test_f1'][np.argmin(best_model_log.cv_results_['rank_test_f1'])])

print(optimal_hyperparameters_2)
print(mean_scores_2)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

...

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

[{'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__penalty': 'none', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 10.0, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 1.0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 100.0, 'classifier__penalty

## Logistic Regression 
    Adult dataset

In [5]:
adult_main = pd.read_csv('adult.data')
adult_test = pd.read_csv ('adult.test')
adult = pd.concat([adult_main,adult_test], ignore_index = True)
#make class column binary
adult_ = adult.replace(' >50K', -1)
adult = adult_.replace(' <=50K', +1)

#move class column to be in 0th index
adult = adult[[ ' <=50K','39', ' State-gov', ' 77516', ' Bachelors', ' 13', ' Never-married',
       ' Adm-clerical', ' Not-in-family', ' White', ' Male', ' 2174', ' 0',
       ' 40', ' United-States', '|1x3 Cross validator']]

#rename columns according to data write-up
adult = adult.rename(columns={" <=50K": "Y", '39': 'age', ' State-gov': 'workclass', ' 77516': 'fnlwgt', 
                              ' Bachelors': 'education', ' 13': 'education-num', ' Never-married': 'marital-status',
                              ' Adm-clerical':'occupation', ' Not-in-family' : 'relationship', ' White' : 'race', 
                              ' Male' : 'sex', ' 2174' : 'capital-gain', ' 0' : 'capital-loss',' 40': 'hours-per-week', 
                              ' United-States' : 'native-country'})

#one hot encode via pd.get_dummies
work = pd.get_dummies(adult['workclass']) 
work.rename(columns = {' ?': 'NA_Work'}, inplace = True)

education = pd.get_dummies(adult['education'])
marital = pd.get_dummies(adult['marital-status'])
occu = pd.get_dummies(adult['occupation']) 
occu.rename(columns = {' ?': 'NA_Occu'}, inplace = True)

relationship = pd.get_dummies(adult['relationship'])
race = pd.get_dummies(adult['race'])
sex = pd.get_dummies(adult['sex'])
country = pd.get_dummies(adult['native-country']) 
country.rename(columns = {' ?': 'NA_Country'}, inplace = True)\

adult_= pd.concat([work,education,marital,occu,relationship,race,sex,country], axis = 1)

#combine the continuous and categorical (now one hot encoded variables)
adult_ = pd.concat([adult[['Y', 'age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']],adult_], axis =1)

adult_ = adult_[adult_.NA_Work == 0]
adult_ = adult_[adult_.NA_Occu == 0]
adult_ = adult_[adult_.NA_Country == 0]

adult_.drop(['NA_Work','NA_Occu','NA_Country'],axis = 1, inplace=True)

adult_.reset_index(inplace = True)
adult_.dropna(inplace = True)

#split off class data into seperate grouping
X_3 = adult_
Y_3 = X_3[['Y']]
X_3 = X_3.iloc[:, 1:]

In [6]:
optimal_hyperparameters_3 = []
mean_scores_3 = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_3, Y_3, train_size = 5000)
    
            # Fit grid search
    best_model_log = clf_log.fit(X_train, y_train)

           
            # output best hyperparameter set indexed at best metric scores
    h1 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_accuracy']) ]    
    h2 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_roc_auc']) ]
    h3 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_f1']) ]
    
    optimal_hyperparameters_3.append(h1)
    optimal_hyperparameters_3.append(h2)
    optimal_hyperparameters_3.append(h3)
    mean_scores_3.append(best_model_log.cv_results_['mean_test_accuracy'][np.argmin(best_model_log.cv_results_['rank_test_accuracy'])])
    mean_scores_3.append(best_model_log.cv_results_['mean_test_roc_auc'][np.argmin(best_model_log.cv_results_['rank_test_roc_auc'])])
    mean_scores_3.append(best_model_log.cv_results_['mean_test_f1'][np.argmin(best_model_log.cv_results_['rank_test_f1'])])

print(optimal_hyperparameters_3)
print(mean_scores_3)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

...

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

[{'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier

## Logistic Regression 
    Cov-type dataset

In [7]:
#read in data move the class column to the 0th column
#rename columns (last 44 columns = leave one out arrangement)
cov_type = pd.read_csv('covtype.data')
cov_type = cov_type[['5', '2596', '51', '3', '258', '0', '510', '221', '232', '148', '6279', '1',
       '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.10',
       '0.11', '0.12', '0.13', '0.14', '0.15', '0.16', '0.17', '0.18', '0.19',
       '0.20', '0.21', '0.22', '0.23', '0.24', '0.25', '0.26', '0.27', '0.28',
       '0.29', '0.30', '0.31', '1.1', '0.32', '0.33', '0.34', '0.35', '0.36',
       '0.37', '0.38', '0.39', '0.40', '0.41', '0.42']]
cov_type.rename(columns = {'5':'Y', '2596':'Elevation', '51': 'Aspect', '3': 'Slope', '258': 
                           'Horizontal_Distance_To_Hydrology', '0': 'Vertical_Distance_To_Hydrology', '510': 
                           'Horizontal_Distance_To_Roadways', '221' : 'Hillshade_9am', '232' : 'Hillshade_Noon', 
                           '148': 'Hillshade_3pm ', '6279': 'Horizontal_Distance_To_Fire_Points'}, inplace = True)

#make the classes binary so the largest class (2) is +1, rest is -1
cov_type = cov_type.replace(to_replace = {'Y': {1: -1}})
cov_type = cov_type.replace(to_replace = {'Y': {3: -1}})
cov_type = cov_type.replace(to_replace = {'Y': {4: -1}})
cov_type = cov_type.replace(to_replace = {'Y': {5: -1}})
cov_type = cov_type.replace(to_replace = {'Y': {6: -1}})
cov_type = cov_type.replace(to_replace = {'Y': {7: -1}})
cov_type = cov_type.replace(to_replace = {'Y': {2: +1}})

X_4 = cov_type
Y_4 = X_4[['Y']]
X_4 = X_4.iloc[:, 1:]

In [8]:
optimal_hyperparameters_4 = []
mean_scores_4 = []

for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_4, Y_4, train_size = 5000)
    
            # Fit grid search
    best_model_log = clf_log.fit(X_train, y_train)

           
            # output best hyperparameter set indexed at best metric scores
    h1 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_accuracy']) ]    
    h2 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_roc_auc']) ]
    h3 = best_model_log.cv_results_['params'][ np.argmin(best_model_log.cv_results_['rank_test_f1']) ]
    
    optimal_hyperparameters_4.append(h1)
    optimal_hyperparameters_4.append(h2)
    optimal_hyperparameters_4.append(h3)
    mean_scores_4.append(best_model_log.cv_results_['mean_test_accuracy'][np.argmin(best_model_log.cv_results_['rank_test_accuracy'])])
    mean_scores_4.append(best_model_log.cv_results_['mean_test_roc_auc'][np.argmin(best_model_log.cv_results_['rank_test_roc_auc'])])
    mean_scores_4.append(best_model_log.cv_results_['mean_test_f1'][np.argmin(best_model_log.cv_results_['rank_test_f1'])])

print(optimal_hyperparameters_4)
print(mean_scores_4)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

...

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return

[{'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 0.01, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}, {'classifier': LogisticRegression(max_iter=5000), 'classifier__C': 10.0, '

In [10]:
print(mean_scores_1)
print(mean_scores_2)
print(mean_scores_3)
print(mean_scores_4)

[0.9634, 0.8408456474856212, 0.9634, 0.9629999999999999, 0.8585445258342455, 0.9629999999999999, 0.9628, 0.8626520803599013, 0.9628, 0.9623999999999999, 0.8638893803183031, 0.9623999999999999, 0.9644, 0.8524666882684103, 0.9644]
[0.7197409409409409, 0.8079984334851341, 0.7197409409409409, 0.7273455455455455, 0.8137284636101553, 0.7273455455455455, 0.7287461461461462, 0.819373954693854, 0.7287461461461462, 0.7283469469469469, 0.8107588243949992, 0.728346946946947, 0.7263427427427428, 0.8166417611893586, 0.7263427427427428]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.7539999999999999, 0.8289365333429334, 0.7540000000000001, 0.7545999999999999, 0.8223653138363348, 0.7545999999999999, 0.7472, 0.8135720700493938, 0.7472, 0.7434000000000001, 0.816724964691866, 0.7434000000000001, 0.7524, 0.8162914606633708, 0.7524]
